In [1]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from ipywidgets import interact, widgets
import numpy as np
import cv2 as cv

In [16]:
image = cv.imread('./cat.png')

table_width = 180
table_height = 230

height, width, _ = image.shape
aspect_ratio = width / height

# scale using width
new_width = int(table_width * (1 - table_width/width))
new_height = int(new_width/aspect_ratio)

print(f'Aspect ratio: {aspect_ratio}')
print(f'New dimensions: ({new_width}, {new_height})')

# scale image to size of table
image = cv.resize(image, (new_width, new_height), interpolation=cv.INTER_AREA)

Aspect ratio: 0.75
New dimensions: (169, 225)


In [17]:
@interact(min_threshold=100, max_threshold=200, aperture=(3,7,2))
def get_contours(min_threshold, max_threshold, aperture=3):
	# get edges using Canny edge detector
	global edges
	edges = cv.Canny(image, min_threshold, max_threshold, apertureSize=aperture)

	# display original image
	plt.rcParams["figure.figsize"]=15,15
	plt.subplot(121)
	# opencv stores image as bgr, need to convert back to display properly
	plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
	plt.title('Original Image') 
	plt.xticks([])
	plt.yticks([])

	# display detected edges
	plt.subplot(122)
	plt.imshow(edges, cmap='gray')
	plt.title('Edges') 
	plt.xticks([])
	plt.yticks([])

	plt.show()


interactive(children=(IntSlider(value=100, description='min_threshold', max=300, min=-100), IntSlider(value=20…

In [24]:
# TODO: RETR_EXTERNAL might be good
contours, hierarchy = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
hierarchy = hierarchy[0]

# hierarchy format: [next (same level), previous (same level), first_child, parent]
# roots have no parent (parent is -1). Get index for all roots
roots = [i for i, h in enumerate(hierarchy) if h[3] == -1]

w = widgets.IntSlider(value=-1, min=-1, max=len(contours))
@interact(cur_depth=w)
def draw_image(cur_depth):
	# display detected contours
	im_contours = image.copy()
	cmap = plt.cm.get_cmap('hsv', len(contours))

	def draw_hierarchy(root, depth, group_index):
		group_color = np.multiply(cmap(group_index), 255)
		# group_color = np.multiply(cmap(depth), 255)

		cur_hierarchy = hierarchy[root]

		contour = contours[root]
		if cur_depth == -1 or depth == cur_depth:
			cv.drawContours(im_contours, [contour], -1, group_color, 1)
			for c in contour:
				cv.circle(im_contours, c[0], 1, group_color, cv.FILLED)

		# draw children of this contour
		child_contour = cur_hierarchy[2]
		if (child_contour != -1):
			draw_hierarchy(child_contour, depth + 1, group_index)

		# draw other contours on current level
		cur_contour = cur_hierarchy[0]
		if (cur_contour != -1):
			draw_hierarchy(cur_contour, depth, group_index)


	for i, group in enumerate(roots):
		draw_hierarchy(group, 0, i)	
	# draw_hierarchy(roots[0], 0)	

	# for i, group in enumerate(contours):
	# 	group_color = np.multiply(cmap(i), 255)
	# 	cv.drawContours(im_contours, [group], -1, group_color, 1)
	# 	# cv.drawContours(im_contours, [group], -1, group_color, cv.FILLED)
	# 	for contour in group:
	# 		cv.circle(im_contours, contour[0], 2, group_color, cv.FILLED)

	plt.imshow(im_contours, cmap='gray')
	plt.title('Contours')
	plt.xticks([])
	plt.yticks([])
	plt.show()

interactive(children=(IntSlider(value=-1, description='cur_depth', max=102, min=-1), Output()), _dom_classes=(…

In [21]:
with open('out.g', 'w') as f:
	f.write('; BEGIN PRE\n')
	f.write('G28\n') # return to start
	f.write('G1 F1000\n') # set print speed
	f.write('M906 X200 Y200\n') # set stepper voltages
	f.write('M808 L0\n') # loop forever
	f.write('; END PRE\n')

	# for i, group in enumerate(contours):
	# 	if i != 1:
	# 		f.write(f'; START GROUP {i}\n')
	# 		for contour in group:
	# 			f.write(f'G01 X{contour[0][0]} Y{contour[0][1]}\n')
	# 		f.write(f'; END GROUP {i}\n')

	def draw_hierarchy(root, depth):
		cur_hierarchy = hierarchy[root]

		contour = contours[root]
		# if depth == 1:
		for point in contour:
			f.write(f'G01 X{point[0][0]} Y{point[0][1]}\n')

		# draw children of this contour
		child_contour = cur_hierarchy[2]
		if (child_contour != -1):
			draw_hierarchy(child_contour, depth + 1)

		# draw other contours on current level
		cur_contour = cur_hierarchy[0]
		if (cur_contour != -1):
			draw_hierarchy(cur_contour, depth)


	for i, group in enumerate(roots):
		f.write(f'; START GROUP {i}\n')
		draw_hierarchy(group, 0)	
		f.write(f'; END GROUP {i}\n')

